In [2]:
# Sudoku AI Solver
# Developed for CSP-based AI Sudoku solving

import copy
from collections import deque

class SudokuAI_Solver:
    def __init__(self, board_file):
        self.board = self.load_board(board_file)
        self.domains = dict()
        self.setup_domains()

    def load_board(self, filename):
        """Load the Sudoku puzzle from a text file."""
        board = dict()
        with open(filename, 'r') as file:
            rows = file.readlines()
            for i in range(9):
                values = list(map(int, rows[i].split()))
                for j in range(9):
                    board[(i, j)] = values[j]
        return board

    def setup_domains(self):
        """Initialize domains for each variable."""
        for cell in self.board:
            if self.board[cell] == 0:
                self.domains[cell] = set(range(1, 10))
            else:
                self.domains[cell] = {self.board[cell]}
    
    def print_board(self, assignment=None):
        """Print the board."""
        if assignment is None:
            assignment = self.board
        for i in range(9):
            row = ''
            for j in range(9):
                value = assignment.get((i, j), 0)
                row += str(value) + ' '
            print(row)

    def neighbors(self, cell):
        """Return all neighboring cells that share a constraint."""
        i, j = cell
        neighbors = set()

        # Same row and column
        for k in range(9):
            if k != j:
                neighbors.add((i, k))
            if k != i:
                neighbors.add((k, j))

        # Same 3x3 subgrid
        box_i = (i // 3) * 3
        box_j = (j // 3) * 3
        for di in range(3):
            for dj in range(3):
                ni, nj = box_i + di, box_j + dj
                if (ni, nj) != (i, j):
                    neighbors.add((ni, nj))

        return neighbors

    def enforce_node_consistency(self):
        """Make each variable node-consistent by removing invalid values."""
        for cell in self.domains:
            if len(self.domains[cell]) > 1:
                to_remove = set()
                for value in self.domains[cell]:
                    for neighbor in self.neighbors(cell):
                        if value == self.board.get(neighbor):
                            to_remove.add(value)
                self.domains[cell] -= to_remove

    def revise(self, x, y):
        """Revise domains to enforce arc consistency between x and y."""
        revised = False
        to_remove = set()
        for value in self.domains[x]:
            if all(value == other for other in self.domains[y]):
                to_remove.add(value)
        if to_remove:
            self.domains[x] -= to_remove
            revised = True
        return revised

    def ac3(self):
        """Apply the AC-3 algorithm to enforce arc consistency on all arcs."""
        queue = deque([(x, y) for x in self.domains for y in self.neighbors(x)])
        while queue:
            (x, y) = queue.popleft()
            if self.revise(x, y):
                if not self.domains[x]:
                    return False
                for z in self.neighbors(x):
                    if z != y:
                        queue.append((z, x))
        return True

    def assignment_complete(self, assignment):
        """Check if assignment is complete."""
        return all(assignment[cell] != 0 for cell in assignment)

    def consistent(self, assignment):
        """Check if assignment is consistent with Sudoku rules."""
        for cell in assignment:
            value = assignment[cell]
            if value == 0:
                continue
            for neighbor in self.neighbors(cell):
                if assignment.get(neighbor, 0) == value:
                    return False
        return True

    def order_domain_values(self, var, assignment):
        """Order domain values using Least Constraining Value (LCV)."""
        lcv = []
        for value in self.domains[var]:
            count = 0
            for neighbor in self.neighbors(var):
                if value in self.domains.get(neighbor, []):
                    count += 1
            lcv.append((count, value))
        lcv.sort()
        return [value for count, value in lcv]

    def select_unassigned_variable(self, assignment):
        """Select unassigned variable using MRV and Degree heuristics."""
        unassigned = [v for v in self.domains if assignment[v] == 0]
        # Minimum Remaining Values (MRV)
        mrv = sorted(unassigned, key=lambda var: (len(self.domains[var]), -len(self.neighbors(var))))
        return mrv[0]

    def backtrack(self, assignment):
        """Backtracking search with inference."""
        if self.assignment_complete(assignment):
            return assignment
        
        var = self.select_unassigned_variable(assignment)
        for value in self.order_domain_values(var, assignment):
            new_assignment = assignment.copy()
            new_assignment[var] = value
            if self.consistent(new_assignment):
                # Inference with AC-3
                saved_domains = copy.deepcopy(self.domains)
                self.domains[var] = {value}
                if self.ac3():
                    result = self.backtrack(new_assignment)
                    if result:
                        return result
                self.domains = saved_domains
        return None

    def solve(self):
        """Solve the Sudoku puzzle."""
        self.enforce_node_consistency()
        self.ac3()
        solution = self.backtrack(self.board)
        if solution:
            print("Solved Sudoku:")
            self.print_board(solution)
        else:
            print("No solution found.")

# Example usage:
# Load and solve easy sudoku
solver = SudokuAI_Solver('sudoku_easy.txt')
solver.solve()

# You can change 'sudoku_easy.txt' to 'sudoku_medium.txt' or 'sudoku_hard.txt' to test others.


Solved Sudoku:
5 3 4 6 7 8 9 1 2 
6 7 2 1 9 5 3 4 8 
1 9 8 3 4 2 5 6 7 
8 5 9 7 6 1 4 2 3 
4 2 6 8 5 3 7 9 1 
7 1 3 9 2 4 8 5 6 
9 6 1 5 3 7 2 8 4 
2 8 7 4 1 9 6 3 5 
3 4 5 2 8 6 1 7 9 


In [10]:
#swap function
solver = SudokuAI_Solver('sudoku_medium.txt')
solver.solve()

Solved Sudoku:
5 8 1 6 7 2 4 3 9 
7 9 2 8 4 3 6 5 1 
3 6 4 5 9 1 7 8 2 
4 3 8 9 5 7 2 1 6 
2 5 6 1 8 4 9 7 3 
1 7 9 3 2 6 8 4 5 
8 4 5 2 1 9 3 6 7 
9 1 3 7 6 8 5 2 4 
6 2 7 4 3 5 1 9 8 
